# INF8215 - Intelligence artif.: méthodes et algorithmes 
## Automne 2019 - TP2 - Recherche Adversarielle 
### Membres de l'équipe
   - Étienne Beauchamp (1741239)
   - Jean Etienne Assaf (1799561)




## Directives de remise
Le travail sera réalisé avec la  même équipe que pour le TP1. Vous remettrez ce fichier nommé TP2\_NomDuMembre1\_NomDuMembre2\_NomDuMembre3.ipynb dans la boîte de remise sur moodle. 

Tout devra être remis avant le **6 novembre à 23h55**. Tout travail en retard sera pénalisé
d’une valeur de 10\% par jour de retard.

## Barème
Partie 1: 25 points

Partie 2: 35 points

Partie 3: 15 points

Partie 4: 15 points

Partie 5: 10 points

Pour un total de 100 points possibles sur 100 points.


## Recherche Adversarielle

La recherche adversarielle est une méthode d’intelligence artificielle qui est utilisée pour prédire les actions d’un adversaire dans un jeu vidéo. Il consiste à construire un arbre avec les actions possibles suivi des réactions de l’adversaire à ces actions. Pour choisir l’action optimale, il existe plusieurs méthodes. Une très commune est la recherche minimax. Elle évalue la valeur des actions et maximise la valeur de celle du joueur et minimise la valeur de celle de l’adversaire.

### Rush Hour

Vous avez vu le jeu de puzzle, Rush Hour, dans le TP précédent. Dans le cadre de ce TP, un adversaire a été ajouté pour transformer Rush Hour en jeu adversarielle. À chaque tour, l’adversaire ajoute une roche dans la grille. Cette roche disparaît après 1 tour et empêche les voitures de se déplacer. Son but est d’obliger le joueur à faire le plus de coups possible et de l’empêcher de bouger complètement. Le joueur peut déplacer une voiture, selon les règles vues dans le tp précédent, par tour. Cette version de Rush Hour est un jeu zero-sum déterministe.

![Rushhour](https://i.stack.imgur.com/NzztF.jpg)

### But

Votre but dans ce TP est de développer un algorithme minimax pour permettre à votre AI de terminer le niveau de Rush Hour **avec un minimun de mouvement**. Une version complétée des classes Rush Hour et State est fournie pour vous aider à faire le TP. Ces classes vont être réutilisées pour représenter les états à évaluer. Vous pouvez les remplacer par vos propres implémentations en Python (vous pouvez aussi réécrire le TP dans le langage de votre choix).  

## 1. Implémentation de Rush Hour (25pts)
Les classes suivantes sont complètes pour représenter les différents états d’une partie de Rush Hour normale.
Il manque la représentation des coups de l’adversaire : vous auriez à implémenter cette partie-là dans les classes touchées.

L’état du stationnement au début et à tout moment du jeu est représenté par un objet de la classe **State** qui contient les champs suivants : 

- `pos` : un vecteur indiquant la position de chaque véhicule sur la grille (la valeur représente la première case occupée par la voiture);

- `c` : l’index de la voiture déplacée pour atteindre l’état courant à partir de l’état précédent;

- `d` : la direction du déplacement fait à partir de l’état précédent pour atteindre l’état courant (+1 : vers la droite ou vers le bas, -1 : vers la gauche ou vers le haut);

- `prev` : l’état précédent

- `score` : le score associé à l'état

- `rock` : la position de la roche dans la grille (ligne,colonne)

In [1]:
import numpy as np
import math
import copy
from collections import deque


class State:
    """
    Contructeur d'un état initial
    """

    def __init__(self, pos):
        """
        pos donne la position de la voiture i dans sa ligne ou colonne (première case occupée par la voiture);
        """
        self.pos = np.array(pos)

        """
        c,d et prev premettent de retracer l'état précédent et le dernier mouvement effectué
        """
        self.c = self.d = self.prev = None

        self.nb_moves = 0
        self.score = 0

        # Position de la roche
        self.rock = []

    """
    Constructeur d'un état à partir du mouvement (c,d)
    """

    def move(self, c, d):
        s = State(self.pos)
        s.prev = self
        s.pos[c] += d
        s.c = c
        s.d = d
        s.nb_moves = self.nb_moves + 1
        s.rock = self.rock
        return s

    def put_rock(self, rock_pos):
        # Nouvel objet State à retourner
        new_s = State(self.pos)
        new_s.prev = self
        new_s.c = self.c
        new_s.d = self.d
        new_s.nb_moves = self.nb_moves + 1
        new_s.score = self.score

        # Ajouter une nouvelle roche et enlever l'ancienne
        new_s.rock = rock_pos

        return new_s

    # Retourne une liste des véhicules qui touche l'endroit où pourrait se déplacer la voiture spécifiée
    def blocking_cars(self, rh, blocked_car_id, blocked_car_dir):
        # Vecteur des véhicules perpendiculaires à la voiture analysée
        v_id = [v_id for v_id in range(1, rh.nbcars) if rh.horiz[v_id] != rh.horiz[blocked_car_id]]

        # Avant et arrière de la voiture analysée
        blocked_car_rear = self.pos[blocked_car_id]
        blocked_car_front = self.pos[blocked_car_id] + rh.length[blocked_car_id] - 1

        # Calcul dépendant de la direction prise par la voiture
        if blocked_car_dir == 1:
            # Vecteur des voitures touchant la ligne de déplacement aurant en face qu'en arrière
            v_in_way = [i for i in v_id if (rh.move_on[i] > blocked_car_front) and
                        (self.pos[i] <= rh.move_on[blocked_car_id]) and
                        (self.pos[i] + rh.length[i] - 1 >= rh.move_on[blocked_car_id])]
        else:
            v_in_way = [i for i in v_id if (rh.move_on[i] < blocked_car_rear) and
                        (self.pos[i] <= rh.move_on[blocked_car_id]) and
                        (self.pos[i] + rh.length[i] - 1 >= rh.move_on[blocked_car_id])]

        # Retourne le vecteur des voitures obstruant le véhicule analysé
        return v_in_way

    # Retourne un si le véhicule est bloqué par la roche
    def blocking_rock(self, rh, car_id, car_dir):
        obstruction = 0

        # Avant et arrière de la voiture analysée
        car_rear = self.pos[car_id]
        car_front = self.pos[car_id] + rh.length[car_id] - 1

        # Dépendant de la direction prise par la voiture
        if rh.move_on[car_id] == self.rock[not rh.horiz[car_id]]:
            if car_dir == 1 and self.rock[rh.horiz[car_id]] == car_front + 1:
                obstruction += 1
            elif car_dir == -1 and self.rock[rh.horiz[car_id]] == car_rear - 1:
                obstruction += 1
        return obstruction

    # Heuristique #3 du premier TP
    def min_number_moves(self, rh):
        nb_moves = 0

        # Liste des voitures face au véhicule rouge
        cars_infront_red = self.blocking_cars(rh, 0, 1)

        # Calcule le nombre strictement minimum requis pour libérer la ligne 2
        for i in cars_infront_red:
            if rh.length[i] == 3:
                nb_moves += 3 - self.pos[i]
            else:
                nb_moves += 1

        return 4 - self.pos[0] + nb_moves

    # Compte le nombre d'obstruction et pondère le poids selon la profondeur de l'obstruction
    def obstruction_count(self, rh, depth):
        nb_obstructions = 0

        # Trouve les voitures qui obstrue la voiture rouge
        obstructed = self.blocking_cars(rh, 0, 1)
        # Nombre d'obstructions
        nb_obstructions += 5*len(obstructed)
        not_yet_visited = []

        # Recherche des obstructions des voitures obstruée qui obstruent... pondéré selon la profondeur de l'obstruction
        for i in range(1, depth):
            # On parcourt toutes les voitures obstruant le passage
            for v in obstructed:
                # Voitures de longueurs 3 bloquant les dernières colonnes
                if i == 1 and rh.length[v] == 3:
                    if rh.move_on[v] == 4:
                        nb_obstructions += len(self.blocking_cars(rh, v, 1))
                    # Les voitures de longueur 3 dans la dernière colonne doivent nécessairement descendre
                    if rh.move_on[v] == 5:
                        # Les obstructions de niveau deux ont un impact important et sont pondérées en conséquence
                        nb_obstructions += 2 * len(self.blocking_cars(rh, v, 1))
                        nb_obstructions -= self.pos[v]  # On récompense le joueur max quand on libère la dernière col
                # On énumère les voitures qui bloquent la voiture bloquée
                not_yet_visited += self.blocking_cars(rh, v, -1)
                not_yet_visited += self.blocking_cars(rh, v, 1)
                # Tient compte des roches
                if self.rock:
                    # Trouve le nombre d'obstruction directe d'une voiture par la roche
                    nb_obstructions += (depth - i) * self.blocking_rock(rh, v, -1)
                    nb_obstructions += (depth - i) * self.blocking_rock(rh, v, 1)

            # Les obstruction plus profondes sont moins importantes
            nb_obstructions += (depth - i) * len(not_yet_visited)
            # On retire les redondances pour l'explorations subséquente
            obstructed = set(not_yet_visited)
            not_yet_visited = []

        #  Retourne le nombre d'obstructions
        return nb_obstructions

    def score_state(self, rh):
        # 10 fois la proximité de la voiture rouge à la sortie accordée
        gain = 10 * self.pos[0]
        # Chaque mouvement et obstruction engendre des pertes
        perte = self.min_number_moves(rh) + self.obstruction_count(rh, 4)
        # Pour encourager la résolution en un nombre minimal de coup
        perte += self.nb_moves
        # Pour éviter de retourner dans un état précédent
        if self.c == self.prev.c and self.d == -self.prev.d:
            perte += 100
        # Affecte la valeur de l'état à son paramètre score
        self.score = gain - perte

    def success(self):
        return self.pos[0] == 4

    def __eq__(self, other):
        if not isinstance(other, State):
            return NotImplemented
        if len(self.pos) != len(other.pos):
            print("les états n'ont pas le même nombre de voitures")

        return np.array_equal(self.pos, other.pos)

    def __hash__(self):
        h = 0
        for i in range(len(self.pos)):
            h = 37 * h + self.pos[i]
        return int(h)

    def __lt__(self, other):
        return (self.score) < (other.score)


La représentation du problème est faite par la classe **Rushhour** qui contient les champs suivants :

- `nbcars` : le nombre de voitures dans la grille;
- `horiz` : un vecteur contenant un bool indiquant si la voiture bouge horizontalement ou pas;
- `length`: un vecteur contenant la longueur de chaque voiture (2 ou 3);
- `move_on` : un vecteur contenant le numéro de la ligne ou la colonne où se trouve la voiture (0-5);
- `color` : un vecteur contenant le string indiquant la couleur de chaque voiture;
- `free_pos` : une matrice 6x6 contenant un bool permettant de savoir si la case est libre

Toutes les informations pour une voiture se trouvent dans le même index dans chacun des vecteurs.

La fonction *`print_pretty_grid()`* sert à imprimer la grille en utilisant la première lettre des couleurs pour vous aider à mieux visualiser le problème.

In [2]:
class Rushhour:

    def __init__(self, horiz, length, move_on, color=None):
        self.nbcars = len(horiz)
        self.horiz = horiz
        self.length = length
        self.move_on = move_on
        self.color = color

        self.free_pos = None

    def init_positions(self, state):
        self.free_pos = np.ones((6, 6), dtype=bool)
        for i in range(self.nbcars):
            if self.horiz[i]:
                self.free_pos[self.move_on[i], state.pos[i]:state.pos[i] + self.length[i]] = False
            else:
                self.free_pos[state.pos[i]:state.pos[i] + self.length[i], self.move_on[i]] = False
        # Emplacement de la roche
        self.free_pos[state.rock] = False

    def possible_moves(self, state):
        self.init_positions(state)
        new_states = []
        for i in range(self.nbcars):
            if self.horiz[i]:
                if state.pos[i] + self.length[i] - 1 < 5 and \
                        self.free_pos[self.move_on[i], state.pos[i] + self.length[i]]:
                    new_states.append(state.move(i, +1))
                if state.pos[i] > 0 and self.free_pos[self.move_on[i], state.pos[i] - 1]:
                    new_states.append(state.move(i, -1))
            else:
                if state.pos[i] + self.length[i] - 1 < 5 and \
                        self.free_pos[state.pos[i] + self.length[i], self.move_on[i]]:
                    new_states.append(state.move(i, +1))
                if state.pos[i] > 0 and self.free_pos[state.pos[i] - 1, self.move_on[i]]:
                    new_states.append(state.move(i, -1))
        return new_states

    def possible_rock_moves(self, state):
        self.init_positions(state)
        new_states = []  # Tous les coups possibles de l'adversaire (avec la roche) à partir de l'état courant

        rock_current_row = rock_current_column = None

        if state.rock:
            (rock_current_row, rock_current_column) = state.rock

        # Ne peut mettre deux roches consécutives sur la même ligne ou sur la ligne 2
        for row in set(range(6)) - {2, rock_current_row}:
            # Ne peut mettre deux roches consécutives sur la même colonne
            for column in set(range(6)) - {rock_current_column}:
                if self.free_pos[row, column]:
                    new_states.append(state.put_rock((row, column)))

        return new_states

    def print_pretty_grid(self, state):
        self.init_positions(state)
        grid = np.chararray((6, 6))
        grid[:] = '-'
        for car in range(self.nbcars):
            for pos in range(state.pos[car], state.pos[car] + self.length[car]):
                if self.horiz[car]:
                    grid[self.move_on[car]][pos] = self.color[car][0]
                else:
                    grid[pos][self.move_on[car]] = self.color[car][0]
        grid[state.rock] = 'x'
        print(grid)


### 1.1 Implémentation

1. Modifier la classe State pour qu'elle contienne la position de la roche ainsi que la fonction *`put_rock()`* pour ajouter une nouvelle roche et enlever l'ancienne. Les roches sont représentées par un tuple (ligne,colonne). La fonction retourne un nouvel objet State.

2. Modifier la fonction *`init_position()`* de la classe Rushhour pour qu'elle tienne compte de l'emplacement des roches.

3. Implémenter la fonction *`possible_rock_moves()`* qui permet de trouver tous les coups possibles de l'adversaire (avec la roche) à partir de l'état courant dans la classe Rushhour. L'adversaire ne peut ni mettre de roche sur la ligne 2 ni mettre deux roches consécutives sur la même ligne ou colonne.

Utiliser la fonction *testRocks()* pour vérifier que vous avez bien pris en compte les roches. Vous devriez avoir ces résultats:

```
État initial
[[b'-' b'-' b'v' b'j' b'j' b'j']
 [b'o' b'-' b'v' b'v' b'b' b'b']
 [b'o' b'r' b'r' b'v' b'-' b'-']
 [b'-' b'r' b'-' b'v' b'v' b'v']
 [b'v' b'r' b'n' b'n' b'-' b'b']
 [b'v' b'b' b'b' b'b' b'-' b'b']]
[[ True  True False False False False]
 [False  True False False False False]
 [False False False False  True  True]
 [ True False  True False False False]
 [False False False False  True False]
 [False False False False  True False]]


Roche 4-4
[[b'-' b'-' b'v' b'j' b'j' b'j']
 [b'o' b'-' b'v' b'v' b'b' b'b']
 [b'o' b'r' b'r' b'v' b'-' b'-']
 [b'-' b'r' b'-' b'v' b'v' b'v']
 [b'v' b'r' b'n' b'n' b'x' b'b']
 [b'v' b'b' b'b' b'b' b'-' b'b']]
[[ True  True False False False False]
 [False  True False False False False]
 [False False False False  True  True]
 [ True False  True False False False]
 [False False False False False False]
 [False False False False  True False]]


Roche 3-2
[[b'-' b'-' b'v' b'j' b'j' b'j']
 [b'o' b'-' b'v' b'v' b'b' b'b']
 [b'o' b'r' b'r' b'v' b'-' b'-']
 [b'-' b'r' b'x' b'v' b'v' b'v']
 [b'v' b'r' b'n' b'n' b'-' b'b']
 [b'v' b'b' b'b' b'b' b'-' b'b']]
[[ True  True False False False False]
 [False  True False False False False]
 [False False False False  True  True]
 [ True False False False False False]
 [False False False False False False]
 [False False False False  True False]]

```

Utiliser la fonction *testPossibleRockMoves()* pour vérifier que vous trouvez bien tous les coups possibles pour l'adversaire. Vous devriez trouver 7 et 3 mouvements possibles.

In [3]:
def testRocks():
    rh = Rushhour([True, False, True, False, False, True, False, True, False, True, False, True],
                 [2, 2, 3, 2, 3, 2, 2, 2, 2, 2, 2, 3],
                 [2, 2, 0, 0, 3, 1, 1, 3, 0, 4, 5, 5],
                 ["rouge", "vert clair", "jaune", "orange", "violet clair", "bleu ciel", "rose", "violet", "vert", "noir", "beige", "bleu"])
    s0 = State([1, 0, 3, 1, 1, 4, 3, 4, 4, 2, 4, 1])
    
    s1= s0.put_rock((4,4))
    s2 = s1.put_rock((3,2)) 
    
    print("État initial")
    rh.print_pretty_grid(s0)
    print(rh.free_pos)
    print('\n')
    
    print("Roche 4-4")
    rh.print_pretty_grid(s1)
    print(rh.free_pos)
    print('\n')
    
    print("Roche 3-2")
    rh.print_pretty_grid(s2)
    print(rh.free_pos)
    print('\n')

testRocks()

État initial
[[b'-' b'-' b'v' b'j' b'j' b'j']
 [b'o' b'-' b'v' b'v' b'b' b'b']
 [b'o' b'r' b'r' b'v' b'-' b'-']
 [b'-' b'r' b'-' b'v' b'v' b'v']
 [b'v' b'r' b'n' b'n' b'-' b'b']
 [b'v' b'b' b'b' b'b' b'-' b'b']]
[[ True  True False False False False]
 [False  True False False False False]
 [False False False False  True  True]
 [ True False  True False False False]
 [False False False False  True False]
 [False False False False  True False]]


Roche 4-4
[[b'-' b'-' b'v' b'j' b'j' b'j']
 [b'o' b'-' b'v' b'v' b'b' b'b']
 [b'o' b'r' b'r' b'v' b'-' b'-']
 [b'-' b'r' b'-' b'v' b'v' b'v']
 [b'v' b'r' b'n' b'n' b'x' b'b']
 [b'v' b'b' b'b' b'b' b'-' b'b']]
[[ True  True False False False False]
 [False  True False False False False]
 [False False False False  True  True]
 [ True False  True False False False]
 [False False False False False False]
 [False False False False  True False]]


Roche 3-2
[[b'-' b'-' b'v' b'j' b'j' b'j']
 [b'o' b'-' b'v' b'v' b'b' b'b']
 [b'o' b'r' b'r' b'v' b'-' b'

In [4]:
def testPossibleRockMoves():
    rh = Rushhour([True, False, True, False, False, True, False, True, False, True, False, True],
                 [2, 2, 3, 2, 3, 2, 2, 2, 2, 2, 2, 3],
                 [2, 2, 0, 0, 3, 1, 1, 3, 0, 4, 5, 5],
                 ["rouge", "vert clair", "jaune", "orange", "violet clair", "bleu ciel", "rose", "violet", "vert", "noir", "beige", "bleu"])
    s = State([1, 0, 3, 1, 1, 4, 3, 4, 4, 2, 4, 1])
    sols = rh.possible_rock_moves(s)
    print(len(sols))
    s1 = s.put_rock((3,4))
    sols = rh.possible_rock_moves(s1)
    print(len(sols))

testPossibleRockMoves()
  

7
3


## 2. Implémentation d'une recherche minimax (35pts)
L'algorithme minimax est l'implémentation la plus commune pour résoudre un jeu adversarielle avec des mouvements limités définis. Cet algorithme est basé sur le principe de construire un arbre avec les différents états possibles selon les actions de chaque joueur (AI et adversaire). L'algorithme veut maximiser les étages où le AI joue et minimiser les étages où l'adversaire joue. Dans ce cas-ci, l'AI déplace les voitures et l'adversaire place les roches. Dans le cadre du TP, l'algorithme n'explore pas jusqu'aux feuilles de l'arbre. Il s'arrête à une profondeur donnée.

La résolution du problème est faite par la classe **MiniMaxSearch** qui contient les champs suivants:

- `rushhour`: la classe Rushhour qui contient les informations sur la grille;
- `state`: la classe State avec l'état actuel de la grille;
- `search_depth`: la profondeur maximale d'exploration de l'algorithme;


In [5]:
import random


class MiniMaxSearch:

    def __init__(self, rushHour, initial_state, search_depth):
        self.rushhour = rushHour
        self.state = initial_state
        self.search_depth = search_depth
        self.visited_states = 0

    def minimax_1(self, current_depth, current_state):
        best_move = (None, None)
        self.visited_states += 1

        # Contient la logique de l'algorithme minimax pour un seul joueur
        if current_depth == 0 or current_state.success():
            current_state.score_state(self.rushhour)
            return current_state.score, (None, None)

        max_value = float('-inf')

        for s in self.rushhour.possible_moves(current_state):
            eval_child, move_child = self.minimax_1(current_depth-1, s)

            if eval_child > max_value:
                max_value = eval_child
                best_move = (s.c, s.d)

        # Retourne le meilleur coup à prendre à partir de l'état courant
        return max_value, best_move

    def minimax_2(self, current_depth, current_state, is_max):
        best_move = (None, None)
        self.visited_states += 1

        # Contient la logique de l'algorithme minimax pour deux joueurs
        if current_depth == 0 or current_state.success():
            current_state.score_state(self.rushhour)
            return current_state.score, (None, None)

        if is_max:
            max_value = float('-inf')
            for s in self.rushhour.possible_moves(current_state):
                eval_child, move_child = self.minimax_2(current_depth - 1, s, False)
                if eval_child > max_value:
                    max_value = eval_child
                    best_move = (s.c, s.d)
            # Retourne le meilleur coup à prendre à partir de l'état courant
            return max_value, best_move

        else:
            min_value = float('inf')
            for s in self.rushhour.possible_rock_moves(current_state):
                eval_child, move_child = self.minimax_2(current_depth - 1, s, True)
                if eval_child < min_value:
                    min_value = eval_child
                    best_move = (s.rock[0], s.rock[1])
            # Retourne le meilleur coup à prendre à partir de l'état courant
            return min_value, best_move

    def minimax_pruning(self, current_depth, current_state, is_max, alpha, beta):
        best_move = (None, None)
        self.visited_states += 1

        # Contient la logique de l'algorithme minimax pour deux joueurs
        if current_depth == 0 or current_state.success():
            current_state.score_state(self.rushhour)
            return current_state.score, (None, None)

        if is_max:
            max_value = float('-inf')
            for s in self.rushhour.possible_moves(current_state):
                eval_child, move_child = self.minimax_pruning(current_depth - 1, s, False, alpha, beta)
                if eval_child > max_value:
                    max_value = eval_child
                    best_move = (s.c, s.d)
                alpha = max(alpha, eval_child)
                if beta <= alpha:
                    break
            # Retourne le meilleur coup à prendre à partir de l'état courant
            return max_value, best_move

        else:
            min_value = float('inf')
            for s in self.rushhour.possible_rock_moves(current_state):
                eval_child, move_child = self.minimax_pruning(current_depth - 1, s, True, alpha, beta)
                if eval_child < min_value:
                    min_value = eval_child
                    best_move = (s.rock[0], s.rock[1])
                beta = min(beta, eval_child)
                if beta <= alpha:
                    break
            # Retourne le meilleur coup à prendre à partir de l'état courant
            return min_value, best_move

    def value_with_probability(self, i, children, vision):
        if vision == "expectimax_aleatoire":
            return random.random() * 1/len(children)
        elif vision == "expectimax_pessimistic":
            # Softmax function
            e_x = np.exp(children[i] - np.max(children))
            return e_x / np.sum(np.exp(children))
        elif vision == "expectimax_optimistic":
            # Softmax inverse function
            e_x = np.exp(-1/(children[i] - np.max(children)))
            sum = 0
            for c in np.arange(len(children)):
                sum += np.exp(-1/children[c])
            if sum != 0:
                return e_x / sum
            else:
                return 0

    def expectimax(self, current_depth, current_state, is_max, vision):
        best_move = (None, None)
        self.visited_states += 1

        # Contient la logique de l'algorithme minimax pour deux joueurs
        if current_depth == 0 or current_state.success():
            current_state.score_state(self.rushhour)
            return current_state.score, (None, None)

        if is_max:
            max_value = float('-inf')
            for s in self.rushhour.possible_moves(current_state):
                eval_child, move_child = self.expectimax(current_depth - 1, s, False, vision)
                if eval_child > max_value:
                    max_value = eval_child
                    best_move = (s.c, s.d)
                # Retourne le meilleur coup à prendre à partir de l'état courant
            return max_value, best_move

        else:
            children = []
            for s in self.rushhour.possible_rock_moves(current_state):
                eval_child, move_child = self.expectimax(current_depth - 1, s, True, vision)
                children.append(eval_child)
            i_state = 0
            min_value = float('inf')
            children_return = 0
            for s in self.rushhour.possible_rock_moves(current_state):
                v = self.value_with_probability(i_state, children, vision)
                if v < min_value:
                    children_return = children[i_state]
                    min_value = v
                    best_move = (s.rock[0], s.rock[1])
                i_state += 1
            return children_return, best_move

    def decide_best_move_1(self):
        # Trouve et exécute le meilleur coup pour une partie à un joueur
        _, best_move = self.minimax_1(self.search_depth, self.state)
        self.state = self.state.move(best_move[0], best_move[1])

    def decide_best_move_2(self, is_max):
        # Trouve et exécute le meilleur coup pour une partie à deux joueurs
        _, best_move = self.minimax_2(self.search_depth, self.state, is_max)
        if is_max:
            self.state = self.state.move(best_move[0], best_move[1])
        else:
            self.state = self.state.put_rock((best_move[0], best_move[1]))

    def decide_best_move_pruning(self, is_max):
        _, best_move = self.minimax_pruning(self.search_depth, self.state, is_max, float('-inf'), float('inf'))
        if is_max:
            self.state = self.state.move(best_move[0], best_move[1])
        else:
            self.state = self.state.put_rock((best_move[0], best_move[1]))

    def decide_best_move_expectimax(self, is_max, vision):
        _, best_move = self.expectimax(self.search_depth, self.state, is_max, vision)
        if is_max:
            self.state = self.state.move(best_move[0], best_move[1])
        else:
            self.state = self.state.put_rock((best_move[0], best_move[1]))

    def solve(self, is_singleplayer, second_player):
        #second_player peut prendre cinq valeurs: 'pessimistic', 'pruning', 'expectimax_aleatoire', 'expectimax_pessimistic', 'expectimax_optimistic'
        # Résout un problème de Rush Hour avec le nombre minimal de coups
        if is_singleplayer:
            while not self.state.success():
                self.decide_best_move_1()
                self.print_move(True, self.state)
                self.solve(True, False)

        elif not is_singleplayer:
            turn = not self.state.nb_moves % 2  # Tours pairs: joueur max
            while not self.state.success():
                if second_player == 'pessimistic':
                    self.decide_best_move_2(turn)
                    self.print_move(turn, self.state)
                    self.solve(False, 'pessimistic')
                elif second_player == 'pruning':
                    self.decide_best_move_pruning(turn)
                    self.print_move(turn, self.state)
                    self.solve(False, 'pruning')
                elif second_player == 'expectimax_aleatoire':
                    self.decide_best_move_expectimax(turn, second_player)
                    self.print_move(turn, self.state)
                    self.solve(False, 'expectimax_aleatoire')
                elif second_player == 'expectimax_optimistic':
                    self.decide_best_move_expectimax(turn, second_player)
                    self.print_move(turn, self.state)
                    self.solve(False, 'expectimax_optimistic')
                elif second_player == 'expectimax_pessimistic':
                    self.decide_best_move_expectimax(turn, second_player)
                    self.print_move(turn, self.state)
                    self.solve(False, 'expectimax_pessimistic')

            return self.visited_states

    def print_move(self, is_max, state):
        # État sous le contrôle de l’agent
        if is_max:
            color = self.rushhour.color[state.c]
            if self.rushhour.horiz[state.c]:
                if state.d == 1:
                    direction = "la droite"
                else:
                    direction = "la gauche"
            else:
                if state.d == 1:
                    direction = "le bas"
                else:
                    direction = "le haut"

            # Imprime le coup fait
            print("%i. Voiture %s vers %s" % (self.state.nb_moves, color, direction))

        # État sous le contrôle de l’adversaire
        else:
            if state.rock:
                # Imprime le coup fait
                print("%i. Roche dans la case %i-%i" % (self.state.nb_moves, state.rock[0], state.rock[1]))


### 2.1 Implémentation simple
1. Implémenter la fonction *`score_state()`* de la classe **State**. Elle affecte la valeur de l'état à son paramètre score. L'état n'est pas nécessairement final. Utiliser l'heuristique qui vous semble la plus pertinente.

2. Implémenter la fonction  *`minimax_1()`*. Cette fonction contient la logique de l'algorithme minimax pour un seul joueur et retourne le meilleur coup à prendre à partir de l'état courant.

3. Implémenter la fonction *`decide_best_move_1()`*. Cette fonction trouve et exécute le meilleur coup pour une partie à un joueur.

4. Implémenter la fonction *`solve()`*. Cette fonction résout un problème de Rush Hour avec le nombre minimal de coups.

4. Implémenter la fonction *`print_move()`* pour imprimer le coup fait. Ex. "Voiture orange vers le haut" ou "Roche dans la case 2-1".

**N.B**: Vous pouvez modifier les déclarations des fonctions ainsi qu'ajouter des fonctions tant que l'algorithme fonctionne.

Utiliser la fonction _test_print_move()_ pour vérifier votre implémentation de _print_move()_. 

In [6]:
def test_print_move():
    rh = Rushhour([True], [2], [2], ["rouge"])
    s = State([0])
    s = s.put_rock((3,1)) # Roche dans la case 3-1
    s = s.move(0, 1) # Voiture rouge vers la droite

    algo = MiniMaxSearch(rh, s, 1)
    algo.print_move(True, s)
    algo.print_move(False, s)

test_print_move()

2. Voiture rouge vers la droite
2. Roche dans la case 3-1


Pour vous aider dans l'implémentation de votre heuristique, 3 exemples de problèmes de Rush Hour sont fournis.

In [7]:
# Solution optimale: 9 moves
rh = Rushhour([True, False, False, False, True],
                 [2, 3, 2, 3, 3],
                 [2, 4, 5, 1, 5],
                 ["rouge", "vert", "bleu", "orange", "jaune"])
s = State([1, 0, 1, 3, 2])
algo = MiniMaxSearch(rh, s,1) 
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve(True, False)
%time

[[ True  True  True  True False  True]
 [ True  True  True  True False False]
 [ True False False  True False False]
 [ True False  True  True  True  True]
 [ True False  True  True  True  True]
 [ True False False False False  True]]
1. Voiture rouge vers la droite
2. Voiture bleu vers le haut
3. Voiture orange vers le haut
4. Voiture jaune vers la gauche
5. Voiture vert vers le bas
6. Voiture vert vers le bas
7. Voiture vert vers le bas
8. Voiture rouge vers la droite
9. Voiture rouge vers la droite
Wall time: 0 ns


In [8]:
# solution optimale: 16 moves
rh = Rushhour([True, True, False, False, True, True, False, False],
                 [2, 2, 3, 2, 3, 2, 3, 3],
                 [2, 0, 0, 0, 5, 4, 5, 3],
                 ["rouge", "vert", "mauve", "orange", "emeraude", "lime", "jaune", "bleu"])
s = State([1, 0, 1, 4, 2, 4, 0, 1])
algo = MiniMaxSearch(rh, s, 1) 
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve(True, False) 
%time

[[False False  True  True  True False]
 [False  True  True False  True False]
 [False False False False  True False]
 [False  True  True False  True  True]
 [False  True  True  True False False]
 [False  True False False False  True]]
1. Voiture lime vers la gauche
2. Voiture jaune vers le bas
3. Voiture vert vers la droite
4. Voiture mauve vers le haut
5. Voiture orange vers le haut
6. Voiture emeraude vers la gauche
7. Voiture emeraude vers la gauche
8. Voiture lime vers la gauche
9. Voiture lime vers la gauche
10. Voiture jaune vers le bas
11. Voiture jaune vers le bas
12. Voiture bleu vers le bas
13. Voiture bleu vers le bas
14. Voiture rouge vers la droite
15. Voiture rouge vers la droite
16. Voiture rouge vers la droite
Wall time: 0 ns


In [9]:
# solution optimale: 14 moves
rh = Rushhour([True, False, True, False, False, False, True, True, False, True, True],
                 [2, 2, 3, 2, 2, 3, 3, 2, 2, 2, 2],
                 [2, 0, 0, 3, 4, 5, 3, 5, 2, 5, 4],
                 ["rouge", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10"])
s = State([0, 0, 3, 1, 2, 1, 0, 0, 4, 3, 4])
algo = MiniMaxSearch(rh, s,1)
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve(True, False)
%time

[[False  True  True False False False]
 [False  True  True False  True False]
 [False False  True False False False]
 [False False False  True False False]
 [ True  True False  True False False]
 [False False False False False  True]]
1. Voiture rouge vers la droite
2. Voiture 1 vers le bas
3. Voiture 2 vers la gauche
4. Voiture 2 vers la gauche
5. Voiture 2 vers la gauche
6. Voiture 3 vers le haut
7. Voiture rouge vers la droite
8. Voiture 10 vers la gauche
9. Voiture 4 vers le haut
10. Voiture 4 vers le haut
11. Voiture rouge vers la droite
12. Voiture 5 vers le bas
13. Voiture 5 vers le bas
14. Voiture rouge vers la droite
Wall time: 0 ns


### 2.2 Implémentation adversarielle

1. Implémenter la fonction  *`minimax_2()`*. Cette fonction contient la logique de l'algorithme minimax pour deux joueurs et retourne le meilleur coup à prendre à partir de l'état courant.

2. Implémenter la fonction *`decide_best_move_2()`*. Cette fonction trouve et exécute le meilleur coup pour une partie à deux joueurs.

3. Modifier la fonction *`solve()`* pour qu'elle puisse résoudre des problèmes à deux joueurs.

4. Modifier la fonction *`score_state()`* de la classe **State** si elle ne tient pas déjà compte des roches.

**N.B**: Vous pouvez modifier les déclarations des fonctions ainsi qu'ajouter des fonctions tant que l'algorithme fonctionne.

 Le nombre donné de coups pour les 3 exemples suivants correspond à une partie sans adversaire. Le nombre de coups de plus dépend de l'implémentation de votre heuristique.

In [10]:
# Solution optimale: 9 moves
rh = Rushhour([True, False, False, False, True],
              [2, 3, 2, 3, 3],
              [2, 4, 5, 1, 5],
              ["rouge", "vert", "bleu", "orange", "jaune"])
s = State([1, 0, 1, 3, 2])
algo = MiniMaxSearch(rh, s, 3)
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
visited_states = algo.solve(False, 'pessimistic')
print("La résolution a nécessité la visite de %i états." % visited_states)
%time

[[ True  True  True  True False  True]
 [ True  True  True  True False False]
 [ True False False  True False False]
 [ True False  True  True  True  True]
 [ True False  True  True  True  True]
 [ True False False False False  True]]
1. Voiture bleu vers le haut
2. Roche dans la case 0-0
3. Voiture rouge vers la droite
4. Roche dans la case 1-1
5. Voiture orange vers le haut
6. Roche dans la case 0-0
7. Voiture vert vers le bas
8. Roche dans la case 5-1
9. Voiture vert vers le bas
10. Roche dans la case 0-0
11. Voiture jaune vers la gauche
12. Roche dans la case 5-4
13. Voiture orange vers le haut
14. Roche dans la case 0-0
15. Voiture vert vers le bas
16. Roche dans la case 1-2
17. Voiture rouge vers la droite
18. Roche dans la case 0-0
19. Voiture rouge vers la droite
La résolution a nécessité la visite de 15984 états.
Wall time: 0 ns


In [11]:
# solution optimale: 16 moves
rh = Rushhour([True, True, False, False, True, True, False, False],
              [2, 2, 3, 2, 3, 2, 3, 3],
              [2, 0, 0, 0, 5, 4, 5, 3],
              ["rouge", "vert", "mauve", "orange", "emeraude", "lime", "jaune", "bleu"])
s = State([1, 0, 1, 4, 2, 4, 0, 1])
algo = MiniMaxSearch(rh, s, 3)
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
visited_states = algo.solve(False, 'pessimistic')
print("La résolution a nécessité la visite de %i états." % visited_states) 
%time

[[False False  True  True  True False]
 [False  True  True False  True False]
 [False False False False  True False]
 [False  True  True False  True  True]
 [False  True  True  True False False]
 [False  True False False False  True]]
1. Voiture lime vers la gauche
2. Roche dans la case 0-2
3. Voiture lime vers la gauche
4. Roche dans la case 4-1
5. Voiture jaune vers le bas
6. Roche dans la case 0-2
7. Voiture lime vers la gauche
8. Roche dans la case 1-1
9. Voiture vert vers la droite
10. Roche dans la case 4-3
11. Voiture mauve vers le haut
12. Roche dans la case 3-0
13. Voiture bleu vers le bas
14. Roche dans la case 0-3
15. Voiture orange vers le haut
16. Roche dans la case 1-1
17. Voiture emeraude vers la gauche
18. Roche dans la case 5-0
19. Voiture jaune vers le bas
20. Roche dans la case 0-3
21. Voiture emeraude vers la gauche
22. Roche dans la case 1-1
23. Voiture bleu vers le bas
24. Roche dans la case 0-3
25. Voiture rouge vers la droite
26. Roche dans la case 1-1
27. Voitu

In [12]:
# solution optimale: 14 moves
rh = Rushhour([True, False, True, False, False, False, True, True, False, True, True],
              [2, 2, 3, 2, 2, 3, 3, 2, 2, 2, 2],
              [2, 0, 0, 3, 4, 5, 3, 5, 2, 5, 4],
              ["rouge", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10"])
s = State([0, 0, 3, 1, 2, 1, 0, 0, 4, 3, 4])
algo = MiniMaxSearch(rh, s, 3)
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
visited_states = algo.solve(False, 'pessimistic')
print("La résolution a nécessité la visite de %i états." % visited_states)
%time

[[False  True  True False False False]
 [False  True  True False  True False]
 [False False  True False False False]
 [False False False  True False False]
 [ True  True False  True False False]
 [False False False False False  True]]
1. Voiture rouge vers la droite
2. Roche dans la case 0-1
3. Voiture 1 vers le bas
4. Roche dans la case 1-2
5. Voiture 2 vers la gauche
6. Roche dans la case 0-1
7. Voiture 4 vers le haut
8. Roche dans la case 1-2
9. Voiture 2 vers la gauche
10. Roche dans la case 0-4
11. Voiture 2 vers la gauche
12. Roche dans la case 1-1
13. Voiture 3 vers le haut
14. Roche dans la case 0-4
15. Voiture rouge vers la droite
16. Roche dans la case 1-1
17. Voiture 4 vers le haut
18. Roche dans la case 3-3
19. Voiture rouge vers la droite
20. Roche dans la case 1-1
21. Voiture 10 vers la gauche
22. Roche dans la case 3-3
23. Voiture 5 vers le bas
24. Roche dans la case 5-5
25. Voiture 6 vers la droite
26. Roche dans la case 1-1
27. Voiture 5 vers le bas
28. Roche dans la c

## 3. Élagage Alpha-Beta (15pts)
Pour accélérer la prise de décision, un élagage est effectué sur l'arbre. Cela permet de ne pas parcourir tout l'arbre. L'élagage alpha-beta est l'implémentation la plus fréquente pour résoudre ce genre de problème. En passant des valeurs d’alpha et de bêta récursivement, l'algorithme connaît la meilleure valeur trouvée à date. Les branches jugées non intéressantes, c'est-à-dire qu'elles ne peuvent pas produire un meilleur résultat que celui déjà trouvé, sont coupées dans le processus.

 
### 3.1 Implémentation

1. Implémenter la fonction *`minimax_pruning()`* pour qu'elle n'explore pas des branches inutiles lors d'une partie à deux joueurs.
2. Implémenter la fonction *`decide_best_move_pruning()`*. Cette fonction trouve et exécute le meilleur coup pour une partie à deux joueurs.
3. Modifier la fonction *`solve()`* pour qu'elle utilise cette nouvelle méthode.
4. Vérifier que cette nouvelle méthode visite bel et bien moins de noeuds. **Montrer cette information dans un tableau**.

In [13]:
# solution optimale: 9 moves
rh = Rushhour([True, False, False, False, True],
              [2, 3, 2, 3, 3],
              [2, 4, 5, 1, 5],
              ["rouge", "vert", "bleu", "orange", "jaune"])
s = State([1, 0, 1, 3, 2])
algo = MiniMaxSearch(rh, s, 3)
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
visited_states = algo.solve(False, 'pruning')
print("La résolution a nécessité la visite de %i états." % visited_states)
%time

[[ True  True  True  True False  True]
 [ True  True  True  True False False]
 [ True False False  True False False]
 [ True False  True  True  True  True]
 [ True False  True  True  True  True]
 [ True False False False False  True]]
1. Voiture bleu vers le haut
2. Roche dans la case 0-0
3. Voiture rouge vers la droite
4. Roche dans la case 1-1
5. Voiture orange vers le haut
6. Roche dans la case 0-0
7. Voiture vert vers le bas
8. Roche dans la case 5-1
9. Voiture vert vers le bas
10. Roche dans la case 0-0
11. Voiture jaune vers la gauche
12. Roche dans la case 5-4
13. Voiture orange vers le haut
14. Roche dans la case 0-0
15. Voiture vert vers le bas
16. Roche dans la case 1-2
17. Voiture rouge vers la droite
18. Roche dans la case 0-0
19. Voiture rouge vers la droite
La résolution a nécessité la visite de 5560 états.
Wall time: 0 ns


In [14]:
# solution optimale: 16 moves
rh = Rushhour([True, True, False, False, True, True, False, False],
             [2, 2, 3, 2, 3, 2, 3, 3],
             [2, 0, 0, 0, 5, 4, 5, 3],
             ["rouge", "vert", "mauve", "orange", "emeraude", "lime", "jaune", "bleu"])
s = State([1, 0, 1, 4, 2, 4, 0, 1])
algo = MiniMaxSearch(rh, s, 3)
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
visited_states = algo.solve(False, 'pruning')
print("La résolution a nécessité la visite de %i états." % visited_states)
%time

[[False False  True  True  True False]
 [False  True  True False  True False]
 [False False False False  True False]
 [False  True  True False  True  True]
 [False  True  True  True False False]
 [False  True False False False  True]]
1. Voiture lime vers la gauche
2. Roche dans la case 0-2
3. Voiture lime vers la gauche
4. Roche dans la case 4-1
5. Voiture jaune vers le bas
6. Roche dans la case 0-2
7. Voiture lime vers la gauche
8. Roche dans la case 1-1
9. Voiture vert vers la droite
10. Roche dans la case 4-3
11. Voiture mauve vers le haut
12. Roche dans la case 3-0
13. Voiture bleu vers le bas
14. Roche dans la case 0-3
15. Voiture orange vers le haut
16. Roche dans la case 1-1
17. Voiture emeraude vers la gauche
18. Roche dans la case 5-0
19. Voiture jaune vers le bas
20. Roche dans la case 0-3
21. Voiture emeraude vers la gauche
22. Roche dans la case 1-1
23. Voiture bleu vers le bas
24. Roche dans la case 0-3
25. Voiture rouge vers la droite
26. Roche dans la case 1-1
27. Voitu

In [15]:
# solution optimale: 14 moves
rh = Rushhour([True, False, True, False, False, False, True, True, False, True, True],
              [2, 2, 3, 2, 2, 3, 3, 2, 2, 2, 2],
              [2, 0, 0, 3, 4, 5, 3, 5, 2, 5, 4],
              ["rouge", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10"])
s = State([0, 0, 3, 1, 2, 1, 0, 0, 4, 3, 4])
algo = MiniMaxSearch(rh, s, 3)
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
visited_states = algo.solve(False, 'pruning')
print("La résolution a nécessité la visite de %i états." % visited_states)
%time

[[False  True  True False False False]
 [False  True  True False  True False]
 [False False  True False False False]
 [False False False  True False False]
 [ True  True False  True False False]
 [False False False False False  True]]
1. Voiture rouge vers la droite
2. Roche dans la case 0-1
3. Voiture 1 vers le bas
4. Roche dans la case 1-2
5. Voiture 2 vers la gauche
6. Roche dans la case 0-1
7. Voiture 4 vers le haut
8. Roche dans la case 1-2
9. Voiture 2 vers la gauche
10. Roche dans la case 0-4
11. Voiture 2 vers la gauche
12. Roche dans la case 1-1
13. Voiture 3 vers le haut
14. Roche dans la case 0-4
15. Voiture rouge vers la droite
16. Roche dans la case 1-1
17. Voiture 4 vers le haut
18. Roche dans la case 3-3
19. Voiture rouge vers la droite
20. Roche dans la case 1-1
21. Voiture 10 vers la gauche
22. Roche dans la case 3-3
23. Voiture 5 vers le bas
24. Roche dans la case 5-5
25. Voiture 6 vers la droite
26. Roche dans la case 1-1
27. Voiture 5 vers le bas
28. Roche dans la c

**Note:** La comparaison du nombre de noeuds visités demandée au point 4 de la question 3.1 se trouve à la fin de ce notebook, soit dans le tableau comparatif final de la section 5.

## 4. Expectimax (15 pts)
L'expectimax est une variante de la recherche minimax. Elle est utilisée lorsqu'il y a une composante de hasard lors de la prise de décision. Depuis le début du TP, il a été supposé que l'adversaire prenait toujours le coup optimal. Maintenant, l'adversaire va faire un coup aléatoirement. Pour résoudre ce cas-là, la recherche expectimax sera utilisée.

### 4.1 Implémentation 

1. Implémenter la fonction *`minimax_expectimax()`* pour qu'elle implémente l'algorithme de recherche. Considérer une probabilité égale pour tous les coups.
2. Implémenter la fonction *`decide_best_move_expectimax()`*. Cette fonction trouve et exécute le meilleur coup pour une partie avec un adversaire imprévisible.
3. Modifier la fonction *`solve()`* pour qu'elle utilise cette nouvelle méthode.
4. Modifier les valeurs de probabilités associées aux coups pour donner une vision optimiste et une vision pessimiste à l'AI. Comparer la performance du AI entre les 3 visions (aléatoire, optimiste, pessimiste). Attention l'adversaire continue à prendre ses décisions au hasard. **Montrer ces informations dans un tableau et les analyser**. 

In [16]:
# solution optimale: 9 moves
rh = Rushhour([True, False, False, False, True],
              [2, 3, 2, 3, 3],
              [2, 4, 5, 1, 5],
              ["rouge", "vert", "bleu", "orange", "jaune"])
s = State([1, 0, 1, 3, 2])
algo = MiniMaxSearch(rh, s, 3)
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
# visited_states = algo.solve(False, 'expectimax_aleatoire')
# visited_states = algo.solve(False, 'expectimax_pessimistic')
visited_states = algo.solve(False, 'expectimax_optimistic')
print("La résolution a nécessité la visite de %i états." % visited_states)
%time

[[ True  True  True  True False  True]
 [ True  True  True  True False False]
 [ True False False  True False False]
 [ True False  True  True  True  True]
 [ True False  True  True  True  True]
 [ True False False False False  True]]
1. Voiture rouge vers la droite

c:\users\user\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:106: RuntimeWarning: divide by zero encountered in long_scalars
c:\users\user\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:109: RuntimeWarning: divide by zero encountered in long_scalars



2. Roche dans la case 0-0
3. Voiture bleu vers le haut
4. Roche dans la case 1-1
5. Voiture orange vers le haut
6. Roche dans la case 0-0
7. Voiture vert vers le bas
8. Roche dans la case 1-1
9. Voiture vert vers le bas
10. Roche dans la case 0-0
11. Voiture jaune vers la gauche
12. Roche dans la case 1-1
13. Voiture vert vers le bas
14. Roche dans la case 0-0
15. Voiture rouge vers la droite
16. Roche dans la case 1-1
17. Voiture rouge vers la droite
La résolution a nécessité la visite de 13724 états.
Wall time: 0 ns


In [17]:
# solution optimale: 16 moves
rh = Rushhour([True, True, False, False, True, True, False, False],
              [2, 2, 3, 2, 3, 2, 3, 3],
              [2, 0, 0, 0, 5, 4, 5, 3],
              ["rouge", "vert", "mauve", "orange", "emeraude", "lime", "jaune", "bleu"])
s = State([1, 0, 1, 4, 2, 4, 0, 1])
algo = MiniMaxSearch(rh, s, 3)
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
# visited_states = algo.solve(False, 'expectimax_aleatoire')
# visited_states = algo.solve(False, 'expectimax_pessimistic')
visited_states = algo.solve(False, 'expectimax_optimistic')
print("La résolution a nécessité la visite de %i états." % visited_states)
%time

[[False False  True  True  True False]
 [False  True  True False  True False]
 [False False False False  True False]
 [False  True  True False  True  True]
 [False  True  True  True False False]
 [False  True False False False  True]]
1. Voiture lime vers la gauche


c:\users\user\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:106: RuntimeWarning: divide by zero encountered in long_scalars


2. Roche dans la case 0-2
3. Voiture lime vers la gauche
4. Roche dans la case 1-1
5. Voiture lime vers la gauche
6. Roche dans la case 0-2
7. Voiture jaune vers le bas
8. Roche dans la case 1-1
9. Voiture jaune vers le bas
10. Roche dans la case 0-2
11. Voiture jaune vers le bas
12. Roche dans la case 1-1
13. Voiture vert vers la droite
14. Roche dans la case 0-0
15. Voiture emeraude vers la gauche
16. Roche dans la case 1-1
17. Voiture mauve vers le haut
18. Roche dans la case 0-3
19. Voiture orange vers le haut
20. Roche dans la case 1-1
21. Voiture emeraude vers la gauche
22. Roche dans la case 0-3
23. Voiture bleu vers le bas
24. Roche dans la case 1-1
25. Voiture bleu vers le bas
26. Roche dans la case 0-3
27. Voiture rouge vers la droite
28. Roche dans la case 1-1
29. Voiture rouge vers la droite
30. Roche dans la case 0-3
31. Voiture rouge vers la droite
La résolution a nécessité la visite de 15303 états.
Wall time: 0 ns


c:\users\user\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:109: RuntimeWarning: divide by zero encountered in long_scalars


In [18]:
# solution optimale: 14 moves
rh = Rushhour([True, False, True, False, False, False, True, True, False, True, True],
              [2, 2, 3, 2, 2, 3, 3, 2, 2, 2, 2],
              [2, 0, 0, 3, 4, 5, 3, 5, 2, 5, 4],
              ["rouge", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10"])
s = State([0, 0, 3, 1, 2, 1, 0, 0, 4, 3, 4])
algo = MiniMaxSearch(rh, s, 2)
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
# visited_states = algo.solve(False, 'expectimax_aleatoire')
# visited_states = algo.solve(False, 'expectimax_pessimistic')
visited_states = algo.solve(False, 'expectimax_optimistic')
print("La résolution a nécessité la visite de %i états." % visited_states)
%time

[[False  True  True False False False]
 [False  True  True False  True False]
 [False False  True False False False]
 [False False False  True False False]
 [ True  True False  True False False]
 [False False False False False  True]]

c:\users\user\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:106: RuntimeWarning: divide by zero encountered in long_scalars



1. Voiture rouge vers la droite
2. Roche dans la case 0-1
3. Voiture 1 vers le bas
4. Roche dans la case 1-2
5. Voiture 2 vers la gauche
6. Roche dans la case 4-0
7. Voiture 2 vers la gauche
8. Roche dans la case 0-4
9. Voiture 2 vers la gauche
10. Roche dans la case 1-1
11. Voiture 3 vers le haut
12. Roche dans la case 0-4
13. Voiture rouge vers la droite
14. Roche dans la case 1-1
15. Voiture 10 vers la gauche
16. Roche dans la case 0-4
17. Voiture 4 vers le haut
18. Roche dans la case 1-1
19. Voiture 4 vers le haut
20. Roche dans la case 3-3
21. Voiture rouge vers la droite
22. Roche dans la case 0-5
23. Voiture 5 vers le bas
24. Roche dans la case 1-1
25. Voiture 5 vers le bas
26. Roche dans la case 0-5
27. Voiture rouge vers la droite
La résolution a nécessité la visite de 1438 états.
Wall time: 0 ns


c:\users\user\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:109: RuntimeWarning: divide by zero encountered in long_scalars


**Note:** La comparaison du nombre de noeuds visités (performance) demandée au point 4 de la question 4.1 se trouve à la fin de ce notebook, soit dans le tableau comparatif final de la section 5.

## 5. Comparaison (10pts)
Comparez la performance la recherche minimax avec la recherche expectimax lorsque l'adversaire est optimale et lorsque l'adversaire est aléatoire. Pour un adversaire aléatoire, prenez une vision neutre. Roulez plusieurs fois les algorithmes (vous pouvez aussi reprendre les tests du TP1) et notez si c'est une victoire ou une défaire pour l'AI ainsi que le nombre de coups moyen, minimal et maximal. Basez-vous sur le nombre de coups optimaux pour résoudre les problèmes pour déterminer un seuil de victoire et de défaite pour l'AI. Précisez votre seuil. **Mettez le tout dans un tableau et analysez vos résultats**. 

**Tableau récapitulatif du nombre d'états visités selon le mode de jeu choisi**


|               	| 2 Players 	| 2 Players    	| Expectimax 	| Expectimax    | Expectimax 	|
|:-------------:	|:---------:	|:----------:	|------------	|-------------	|:----------:	|
|               	|  Minimax  	| Alpha/Beta 	| Aleatoire  	| Pessimistic 	| Optimistic 	|
|  Test 9 moves 	|    15984    	|     5560    	|    15279.6   	|   15984     	|    13724    	|
| Test 16 moves 	|    18078    	|     7667    	|    18022.2 	|   18078     	|   15303    	|
| Test 14 moves 	|    10950   	|    4153    	|    1581.6 	|   1829     	|    1438    	|

**Note:** Tout d'abord, il est important de mentionner que ces valeurs, pour les 3 visions expectimax de l'AI, sont des moyennes obtenues à partir de 5 roulements de l'algorithme. Cependant, dans les cas pessimiste et optimiste, le nombre d'états visités demeurait identique. Cela s'explique par le fait que la fonction de probabilité utilisée n'est pas suffisamment permissive à l'erreur. Il serait alors intéressant de trouver une meilleur fonction de façon à mieux voir les effets de cette possibilité d'erreur de l'adversaire.

De ce tableau, il est possible de remarquer que l'**élagage alpha/beta** permet bel et bien de réduire le nombre d'états visités. Ici, la réduction est supérieure à 50%; cet algorithme permet de couper plus de la moitié des branches possibles. Cela concorde avec l'objectif de cette méthode consistant à couper les branches dont l'algorithme sait ne pas pouvoir obtenir mieux de celle-ci. L'efficience de recherche est donc améliorée.

Par la suite, la comparaison des résultats de expectimax est relativement plus difficile puisque la performance du joueur dépend des risques qu'il prend en fonction de son adversaire. 

Dans le cas d'un **adversaire aléatoire**, celui-ci ne prend pas toujours les meilleures choix d'opposition. Ainsi, le jeu peut se terminer plus rapidement qu'avec un adversaire parfait (sans erreur), faisant en sortes que le nombre d'états visités est moindre. L'effet est plus marqué dans le test à 14 coups, mais il demeure qu'en moyenne le nombre d'états visités est moindre qu'avec un minimax sans erreur.

Puis, dans le cas de l'**expectimax pessimiste**, le comportement de l'adversaire est similaire à l'adversaire parfait étant le minimax à 2 joueurs. En effet, on s'attend au à un adversaire parfait, mais ce dernier peut commettre quelques erreurs. Ainsi, puisque le jeu se termine plus rapidement que ce que le joueur s'attend, le nombre d'états visités est inférieur au jeu avec un adversaire parfait, mais tout de même près. Tout cela est le résultat qu'on attendait, mais la fonction de probabilité utilisé empêche de confirmer nos attentes. Il demeure que l'adversaire fait moins d'erreurs que l'aléatoire puisque le test a nécessité plus d'états visités que l'aléatoire.

Finalement, lorsque l'on regarde la situation de l'**expectimax optimiste**, l'adversaire effectue plusieurs coups qui ne lui sont pas optimaux, permettant ainsi au joueur se terminer le jeu plus rapidement encore. Cela explique pourquoi il s'agit de l'expectimax avec le moins d'états visités. Le choix de la fonction de probabilité nous demande de relativiser notre interprétation. En fait, ici, on voit que l'adversaire fait davantage d'erreur que dans le cas pessimiste puisqu'on visite moins d'états et cela signifie un jeu se terminant plus vite. 